In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))


Result: 4999999950000000


In [6]:
print(1)

1


In [8]:
import time
import concurrent.futures
from time import sleep
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor


list = [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10 ,11, 12 ,13 ,14, 15, 16, 17]

def print_number(number):
    print (number)

def do_thread_crawl():
    thread_list = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for number in list:
            thread_list.append(executor.submit(print_number, number))
            
        print('submit complete')
        
        for execution in concurrent.futures.as_completed(thread_list):
            execution.result()
            
        print('end?')
        

start_time = time.time()

do_thread_crawl()

print("--- elapsed time %s seconds ---" % (time.time() - start_time))



12

345
6
7
8
9
10
11
12
13


submit complete
1415
16
17

end?
--- elapsed time 0.0076580047607421875 seconds ---


In [14]:
from threading import Thread

def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.append(total)
    return

if __name__ == "__main__":
    START, END = 0, 100000000
    result = []
    th1 = Thread(target=work, args=(1, START, END, result))
    
    th1.start()
    th1.join()

    print(f"Result: {sum(result)}")


Result: 4999999950000000
